# <p style="text-align:center"> **GIAI ĐOẠN 4: XÂY DỰNG VÀ ĐÁNH GIÁ MÔ HÌNH** </p>


### <p style="text-align:center"> **I. IMPORT THƯ VIỆN CẦN THIẾT** </p>

In [48]:
# Import thư viện cần thiết để thao tác dữ liệu
import pandas as pd
import numpy as np 

# Import thư viện để trực quan những kết quả tìm hiểu được:
import matplotlib.pyplot as plt

#Import thư viện hỗ trợ xây dựng mô hình
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score,KFold, RepeatedStratifiedKFold, GridSearchCV, RandomizedSearchCV

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.metrics import mean_squared_error

In [19]:
#Lấy các biến từ pha trước vào
%store -r dfMidfielder
%store -r dfKeeper
%store -r dfForward
%store -r dfDefender

In [20]:
#Lưu trữ các biến lại để tránh bị dính các note ở các giá trị chưa được khai báo 
dfMidfieder = dfMidfielder
dfKeeper = dfKeeper
dfForward = dfForward
dfDefender = dfDefender

### <p style="text-align:center"> **II.XỬ LÍ DỮ LIỆU ĐỂ PHÙ HỢP XÂY DỰNG MÔ HÌNH** </p>

Đầu tiên, vì tính chất của bộ dữ liệu không đồng nhất về các cột đặc điểm, ta sẽ không sử dụng dữ liệu của các thủ môn để xây dựng mô hình. Kế tiếp, vì các đặc điểm của tiền vệ, hậu vệ, tiền đạo cũng không đồng nhất; ta chỉ sử dụng những đặc tính chung của loại cầu thủ đó mà thôi.

In [21]:
#Lấy các thuộc tính giống nhau trong 3 bộ dữ liệu của tiền vệ, tiền đạo và hậu vệ
colMidfielder = dfMidfieder.columns
colForward = dfForward.columns
colDefender = dfDefender.columns

same = [i for i in colMidfielder if i in colForward and i in colDefender]
print(same)

['Player name', 'Appearances', 'Wins', 'Losses', 'Headed goals', 'Goals with right foot', 'Goals with left foot', 'Hit woodwork', 'Assists', 'Passes', 'Big chances created', 'Crosses', 'Yellow cards', 'Red cards', 'Fouls', 'Offsides', 'Tackles', 'Blocked shots', 'Interceptions', 'Clearances']


In [22]:
#Loại bỏ các cột dữ liệu dư thừa
dfDefender = dfDefender[dfDefender.columns.intersection(same)]
dfForward = dfForward[dfForward.columns.intersection(same)]
dfMidfieder = dfMidfieder[dfMidfieder.columns.intersection(same)]

In [23]:
#Thêm một cột vị trí ở cuối cùng
dfDefender['Position'] = 'DF'
dfForward['Position'] = 'FW'
dfMidfieder['Position'] = 'MF'

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_19052\2615243928.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfDefender['Position'] = 'DF'
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_19052\2615243928.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfForward['Position'] = 'FW'
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_19052\2615243928.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [24]:
#Gộp ba bảng lại thành một dataframe thống nhất
df = pd.concat([dfDefender, dfForward, dfMidfieder], ignore_index=True)

In [25]:
#Kiểm tra kích cỡ của bộ dữ liệu
df.shape

(908, 21)

In [26]:
#Loại bỏ cột tên (không cần thiết cho việc xây dựng mô hình)
df.drop(columns = 'Player name', inplace=True)

Đưa dữ liệu về dạng trung bình mỗi trận đấu để khoảng cách giữa dữ liệu của các cầu thủ mới chơi
và các cầu thủ lâu năm không bị quá lệch nhau

In [27]:
#Cầu thủ chưa ra sân nhưng lại có dữ liệu thi đấu (lỗi của data)
df.drop([723], inplace = True)


In [28]:
y = df['Position']
y.replace({'FW': 1, 'MF': 2, 'DF': 3}, inplace = True)
df.drop(columns='Position', inplace=True)

In [29]:
new_df = df.iloc[:,1:].apply(lambda x: round(x / df.Appearances, 2))
new_df.dtypes

Wins                     float64
Losses                   float64
Tackles                  float64
Blocked shots            float64
Interceptions            float64
Clearances               float64
Assists                  float64
Passes                   float64
Big chances created      float64
Crosses                  float64
Yellow cards             float64
Red cards                float64
Fouls                    float64
Offsides                 float64
Headed goals             float64
Goals with right foot    float64
Goals with left foot     float64
Hit woodwork             float64
dtype: object

In [30]:
#FILL NA BY MEAN
new_df.fillna(round(new_df.mean(), 2), inplace=True)
new_df
    

,Wins,Losses,Tackles,Blocked shots,Interceptions,Clearances,Assists,Passes,Big chances created,Crosses,Yellow cards,Red cards,Fouls,Offsides,Headed goals,Goals with right foot,Goals with left foot,Hit woodwork
0,0.39,0.40,1.18,0.23,0.70,1.28,0.06,28.00,0.09,1.15,0.11,0.0,0.71,0.11,0.01,0.04,0.03,0.02
1,0.21,0.44,1.12,0.07,1.23,5.60,0.00,55.44,0.00,0.07,0.02,0.0,0.51,0.05,0.02,0.00,0.00,0.00
2,0.39,0.40,1.18,0.23,0.70,1.28,0.06,28.00,0.09,1.15,0.11,0.0,0.71,0.11,0.01,0.04,0.03,0.02
3,0.39,0.40,1.18,0.23,0.70,1.28,0.06,28.00,0.09,1.15,0.11,0.0,0.71,0.11,0.01,0.04,0.03,0.02
4,0.00,1.00,1.00,0.00,1.00,1.00,0.00,49.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903,0.47,0.27,0.78,0.67,0.45,0.22,0.12,26.88,0.29,3.47,0.12,0.0,0.86,0.18,0.00,0.00,0.12,0.04
904,0.63,0.25,1.11,0.63,0.24,0.32,0.13,41.38,0.21,2.00,0.10,0.0,0.63,0.06,0.02,0.03,0.17,0.03
905,0.27,0.42,1.69,0.28,1.94,1.21,0.05,33.68,0.03,0.92,0.18,0.0,1.04,0.03,0.00,0.01,0.01,0.01
906,0.43,0.35,1.67,0.18,1.37,0.51,0.10,51.49,0.10,2.02,0.10,0.0,0.86,0.00,0.00,0.00,0.00,0.02


## PHÂN CHIA TẬP TRAIN VÀ TẬP TEST

KÍCH THƯỚC TẬP TEST LÀ 20%, random state tùy ý

In [31]:
X_train, X_test, y_train, y_test = train_test_split(new_df, y, test_size=0.3, random_state=123)

### SGD Classifier

In [32]:
param_grid = {'loss': ['hinge', 'huber', 'squared_hinge', 'perceptron'], 
            'penalty' : ['l1', 'l2', 'elasticnet'],
            'shuffle' :[True,False], 
            'fit_intercept': [True, False],
            'epsilon': [0.1, 0.5]}
            
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)

sgd_grid = GridSearchCV( SGDClassifier(),param_grid = param_grid,
                          cv=cv, 
                          scoring='neg_mean_squared_error', 
                          verbose=3) 
sgd_grid.fit(X_train,y_train)

Fitting 15 folds for each of 96 candidates, totalling 1440 fits
[CV 1/15] END epsilon=0.1, fit_intercept=True, loss=hinge, penalty=l1, shuffle=True;, score=-0.614 total time=   0.0s
[CV 2/15] END epsilon=0.1, fit_intercept=True, loss=hinge, penalty=l1, shuffle=True;, score=-0.409 total time=   0.0s
[CV 3/15] END epsilon=0.1, fit_intercept=True, loss=hinge, penalty=l1, shuffle=True;, score=-0.850 total time=   0.0s
[CV 4/15] END epsilon=0.1, fit_intercept=True, loss=hinge, penalty=l1, shuffle=True;, score=-0.811 total time=   0.0s
[CV 5/15] END epsilon=0.1, fit_intercept=True, loss=hinge, penalty=l1, shuffle=True;, score=-0.556 total time=   0.0s
[CV 6/15] END epsilon=0.1, fit_intercept=True, loss=hinge, penalty=l1, shuffle=True;, score=-0.567 total time=   0.0s
[CV 7/15] END epsilon=0.1, fit_intercept=True, loss=hinge, penalty=l1, shuffle=True;, score=-0.591 total time=   0.0s
[CV 8/15] END epsilon=0.1, fit_intercept=True, loss=hinge, penalty=l1, shuffle=True;, score=-0.370 total time=

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=5, random_state=None),
             estimator=SGDClassifier(),
             param_grid={'epsilon': [0.1, 0.5], 'fit_intercept': [True, False],
                         'loss': ['hinge', 'huber', 'squared_hinge',
                                  'perceptron'],
                         'penalty': ['l1', 'l2', 'elasticnet'],
                         'shuffle': [True, False]},
             scoring='neg_mean_squared_error', verbose=3)

In [33]:
print('Best Score: %s' % sgd_grid.best_score_)
print('Best Hyperparameters: %s' % sgd_grid.best_params_)

Best Score: -0.5361329833770778
Best Hyperparameters: {'epsilon': 0.1, 'fit_intercept': True, 'loss': 'perceptron', 'penalty': 'l1', 'shuffle': True}


In [34]:
print(classification_report(sgd_grid.predict(X_test), y_test))

              precision    recall  f1-score   support

           1       0.57      0.61      0.59        44
           2       0.47      0.73      0.58        75
           3       0.89      0.64      0.74       154

    accuracy                           0.66       273
   macro avg       0.65      0.66      0.64       273
weighted avg       0.73      0.66      0.67       273



### k-NN Classifier

In [35]:
param_grid = {'n_neighbors': [5, 7, 10], 
            'algorithm' : ['ball_tree', 'kd_tree', 'brute'],
            'leaf_size': [30, 35, 40]}
            
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)

knn_grid = GridSearchCV(KNeighborsClassifier(),param_grid = param_grid,
                          cv=cv, 
                          scoring='neg_mean_squared_error', 
                          verbose=3) 
knn_grid.fit(X_train,y_train)

Fitting 50 folds for each of 27 candidates, totalling 1350 fits
[CV 1/50] END algorithm=ball_tree, leaf_size=30, n_neighbors=5;, score=-0.961 total time=   0.0s
[CV 2/50] END algorithm=ball_tree, leaf_size=30, n_neighbors=5;, score=-0.795 total time=   0.0s
[CV 3/50] END algorithm=ball_tree, leaf_size=30, n_neighbors=5;, score=-1.008 total time=   0.0s
[CV 4/50] END algorithm=ball_tree, leaf_size=30, n_neighbors=5;, score=-0.898 total time=   0.0s
[CV 5/50] END algorithm=ball_tree, leaf_size=30, n_neighbors=5;, score=-0.984 total time=   0.0s
[CV 6/50] END algorithm=ball_tree, leaf_size=30, n_neighbors=5;, score=-0.969 total time=   0.0s
[CV 7/50] END algorithm=ball_tree, leaf_size=30, n_neighbors=5;, score=-1.110 total time=   0.0s
[CV 8/50] END algorithm=ball_tree, leaf_size=30, n_neighbors=5;, score=-0.402 total time=   0.0s
[CV 9/50] END algorithm=ball_tree, leaf_size=30, n_neighbors=5;, score=-0.843 total time=   0.0s
[CV 10/50] END algorithm=ball_tree, leaf_size=30, n_neighbors=5

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=10, n_splits=5, random_state=None),
             estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['ball_tree', 'kd_tree', 'brute'],
                         'leaf_size': [30, 35, 40], 'n_neighbors': [5, 7, 10]},
             scoring='neg_mean_squared_error', verbose=3)

In [36]:
print('Best Score: %s' % knn_grid.best_score_)
print('Best Hyperparameters: %s' % knn_grid.best_params_)

Best Score: -0.8808461442319709
Best Hyperparameters: {'algorithm': 'kd_tree', 'leaf_size': 30, 'n_neighbors': 10}


In [37]:
print(classification_report(knn_grid.predict(X_test), y_test))

              precision    recall  f1-score   support

           1       0.51      0.45      0.48        53
           2       0.37      0.61      0.46        71
           3       0.80      0.59      0.68       149

    accuracy                           0.57       273
   macro avg       0.56      0.55      0.54       273
weighted avg       0.63      0.57      0.58       273



## Random Forest Classifier

In [38]:
param_grid = {'n_estimators' : [100, 150, 200], 
            'criterion' : ["gini", "entropy"],
            'min_samples_split' : [2, 4, 6]}
            
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)

rfc_grid = GridSearchCV(RandomForestClassifier(),param_grid = param_grid,
                          cv=cv, 
                          scoring='neg_mean_squared_error', 
                          verbose=3) 
rfc_grid.fit(X_train,y_train)

Fitting 50 folds for each of 18 candidates, totalling 900 fits
[CV 1/50] END criterion=gini, min_samples_split=2, n_estimators=100;, score=-0.512 total time=   0.2s
[CV 2/50] END criterion=gini, min_samples_split=2, n_estimators=100;, score=-0.449 total time=   0.1s
[CV 3/50] END criterion=gini, min_samples_split=2, n_estimators=100;, score=-0.378 total time=   0.2s
[CV 4/50] END criterion=gini, min_samples_split=2, n_estimators=100;, score=-0.559 total time=   0.2s
[CV 5/50] END criterion=gini, min_samples_split=2, n_estimators=100;, score=-0.548 total time=   0.2s
[CV 6/50] END criterion=gini, min_samples_split=2, n_estimators=100;, score=-0.291 total time=   0.2s
[CV 7/50] END criterion=gini, min_samples_split=2, n_estimators=100;, score=-0.661 total time=   0.1s
[CV 8/50] END criterion=gini, min_samples_split=2, n_estimators=100;, score=-0.378 total time=   0.2s
[CV 9/50] END criterion=gini, min_samples_split=2, n_estimators=100;, score=-0.591 total time=   0.2s
[CV 10/50] END crit

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=10, n_splits=5, random_state=None),
             estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'min_samples_split': [2, 4, 6],
                         'n_estimators': [100, 150, 200]},
             scoring='neg_mean_squared_error', verbose=3)

In [39]:
print('Best Score: %s' % rfc_grid.best_score_)
print('Best Hyperparameters: %s' % rfc_grid.best_params_)

Best Score: -0.4422397200349956
Best Hyperparameters: {'criterion': 'entropy', 'min_samples_split': 2, 'n_estimators': 150}


In [40]:
print(classification_report(rfc_grid.predict(X_test), y_test))

              precision    recall  f1-score   support

           1       0.57      0.69      0.63        39
           2       0.53      0.73      0.61        84
           3       0.87      0.64      0.74       150

    accuracy                           0.67       273
   macro avg       0.66      0.69      0.66       273
weighted avg       0.72      0.67      0.68       273



## AdaBoost Classifier

In [41]:
param_grid = {'n_estimators' : [50, 100, 150], 
            'learning_rate' : [0.4, 0.2, 0.7, 1.0],
            'random_state' : [99, 388, 123]}
            
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)

ab_grid = GridSearchCV(AdaBoostClassifier(),param_grid = param_grid,
                          cv=cv, 
                          scoring='neg_mean_squared_error', 
                          verbose=3) 
ab_grid.fit(X_train,y_train)

Fitting 50 folds for each of 36 candidates, totalling 1800 fits
[CV 1/50] END learning_rate=0.4, n_estimators=50, random_state=99;, score=-0.772 total time=   0.1s
[CV 2/50] END learning_rate=0.4, n_estimators=50, random_state=99;, score=-0.394 total time=   0.0s
[CV 3/50] END learning_rate=0.4, n_estimators=50, random_state=99;, score=-0.386 total time=   0.1s
[CV 4/50] END learning_rate=0.4, n_estimators=50, random_state=99;, score=-0.535 total time=   0.0s
[CV 5/50] END learning_rate=0.4, n_estimators=50, random_state=99;, score=-0.516 total time=   0.0s
[CV 6/50] END learning_rate=0.4, n_estimators=50, random_state=99;, score=-0.433 total time=   0.0s
[CV 7/50] END learning_rate=0.4, n_estimators=50, random_state=99;, score=-0.756 total time=   0.0s
[CV 8/50] END learning_rate=0.4, n_estimators=50, random_state=99;, score=-0.386 total time=   0.0s
[CV 9/50] END learning_rate=0.4, n_estimators=50, random_state=99;, score=-0.386 total time=   0.0s
[CV 10/50] END learning_rate=0.4, n_

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=10, n_splits=5, random_state=None),
             estimator=AdaBoostClassifier(),
             param_grid={'learning_rate': [0.4, 0.2, 0.7, 1.0],
                         'n_estimators': [50, 100, 150],
                         'random_state': [99, 388, 123]},
             scoring='neg_mean_squared_error', verbose=3)

In [42]:
print('Best Score: %s' % ab_grid.best_score_)
print('Best Hyperparameters: %s' % ab_grid.best_params_)

Best Score: -0.4605336832895888
Best Hyperparameters: {'learning_rate': 0.2, 'n_estimators': 50, 'random_state': 99}


In [43]:
print(classification_report(ab_grid.predict(X_test), y_test))

              precision    recall  f1-score   support

           1       0.45      0.64      0.52        33
           2       0.86      0.49      0.62       205
           3       0.28      0.89      0.43        35

    accuracy                           0.56       273
   macro avg       0.53      0.67      0.53       273
weighted avg       0.74      0.56      0.59       273



## Decision Tree Classifier

In [44]:
param_grid = {'criterion': ['gini', 'entropy', 'log_loss'],
                            'splitter': ['best', 'random'],
                            'max_depth': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]} 
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

dt_grid = GridSearchCV(DecisionTreeClassifier(), 
                    param_grid, 
                    cv=cv, 
                    scoring='neg_mean_squared_error', 
                    verbose=3)
                    
dt_grid.fit(X_train, y_train)

Fitting 30 folds for each of 60 candidates, totalling 1800 fits
[CV 1/30] END criterion=gini, max_depth=5, splitter=best;, score=-0.422 total time=   0.0s
[CV 2/30] END criterion=gini, max_depth=5, splitter=best;, score=-0.547 total time=   0.0s
[CV 3/30] END criterion=gini, max_depth=5, splitter=best;, score=-0.422 total time=   0.0s
[CV 4/30] END criterion=gini, max_depth=5, splitter=best;, score=-0.531 total time=   0.0s
[CV 5/30] END criterion=gini, max_depth=5, splitter=best;, score=-0.571 total time=   0.0s
[CV 6/30] END criterion=gini, max_depth=5, splitter=best;, score=-0.333 total time=   0.0s
[CV 7/30] END criterion=gini, max_depth=5, splitter=best;, score=-0.698 total time=   0.0s
[CV 8/30] END criterion=gini, max_depth=5, splitter=best;, score=-0.683 total time=   0.0s
[CV 9/30] END criterion=gini, max_depth=5, splitter=best;, score=-0.667 total time=   0.0s
[CV 10/30] END criterion=gini, max_depth=5, splitter=best;, score=-0.460 total time=   0.0s
[CV 11/30] END criterion=

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=10, random_state=1),
             estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50],
                         'splitter': ['best', 'random']},
             scoring='neg_mean_squared_error', verbose=3)

In [45]:
print('Best Score: %s' % dt_grid.best_score_)
print('Best Hyperparameters: %s' % dt_grid.best_params_)

Best Score: -0.48530092592592594
Best Hyperparameters: {'criterion': 'entropy', 'max_depth': 5, 'splitter': 'random'}


In [46]:
print(classification_report(dt_grid.predict(X_test), y_test))

              precision    recall  f1-score   support

           1       0.45      0.50      0.47        42
           2       0.67      0.48      0.56       162
           3       0.45      0.71      0.55        69

    accuracy                           0.54       273
   macro avg       0.52      0.56      0.53       273
weighted avg       0.58      0.54      0.54       273

